#### GA. NCM

Genetic algorithm application to the NCM

In [1]:
import pandas as pd
import numpy as np
import os 
import random
from Funciones import *
from scipy.spatial.distance import cdist
medida = 'euclidean'

random.seed(42)

import pygad 
from sklearn.mixture import GaussianMixture


### Load the datasets

In [2]:
# Paths and Embeddings for the training set
set = 'Mobio'
a,b = load_train(set)

Img_Con_train_paths = reshape_paths(a[0][0])
Aud_Con_train_paths = reshape_paths(a[0][1])
Mix_Con_train_paths = reshape_paths(a[0][2])

Img_Descon_train_paths = reshape_paths(a[1][0])
Aud_Descon_train_paths = reshape_paths(a[1][1])
Mix_Descon_train_paths = reshape_paths(a[1][2])


Img_Con_train_embeddings = reshape_embed(b[0][0])
Aud_Con_train_embeddings = reshape_embed(b[0][1])
Mix_Con_train_embeddings = reshape_embed(b[0][2])

Img_Descon_train_embeddings = reshape_embed(b[1][0])
Aud_Descon_train_embeddings = reshape_embed(b[1][1])
Mix_Descon_train_embeddings = reshape_embed(b[1][2])


# Paths and Embeddings for the validation set
a,b = load_validation(set)

Img_Con_validation_paths = reshape_paths(a[0][0])
Aud_Con_validation_paths = reshape_paths(a[0][1])
Mix_Con_validation_paths = reshape_paths(a[0][2])

Img_Descon_validation_paths = reshape_paths(a[1][0])
Aud_Descon_validation_paths = reshape_paths(a[1][1])
Mix_Descon_validation_paths = reshape_paths(a[1][2])


Img_Con_validation_embeddings = reshape_embed(b[0][0])
Aud_Con_validation_embeddings = reshape_embed(b[0][1])
Mix_Con_validation_embeddings = reshape_embed(b[0][2])

Img_Descon_validation_embeddings = reshape_embed(b[1][0])
Aud_Descon_validation_embeddings = reshape_embed(b[1][1])
Mix_Descon_validation_embeddings = reshape_embed(b[1][2])


# Paths and Embeddings for the test set
a,b = load_test(set)

Img_Con_test_paths = reshape_paths(a[0][0])
Aud_Con_test_paths = reshape_paths(a[0][1])
Mix_Con_test_paths = reshape_paths(a[0][2])

Img_Descon_test_paths = reshape_paths(a[1][0])
Aud_Descon_test_paths = reshape_paths(a[1][1])
Mix_Descon_test_paths = reshape_paths(a[1][2])

Img_Con_test_embeddings = reshape_embed(b[0][0])
Aud_Con_test_embeddings = reshape_embed(b[0][1])
Mix_Con_test_embeddings = reshape_embed(b[0][2])

Img_Descon_test_embeddings = reshape_embed(b[1][0])
Aud_Descon_test_embeddings = reshape_embed(b[1][1])
Mix_Descon_test_embeddings = reshape_embed(b[1][2])



##
Conocidos = len(Img_Con_train_embeddings)
Desconocidos = len(Img_Descon_train_embeddings)
muestras_img = len(Img_Con_train_embeddings[0])
muestras_aud = len(Aud_Con_train_embeddings[0])

img_test = len(Img_Con_test_embeddings[0])
aud_test = len(Aud_Con_test_embeddings[0])

columnas = a[0][0].columns

In [3]:
print ('Known: ', Conocidos)
print ('Unknown: ', Desconocidos)
print ('Images per person (train): ', muestras_img)
print ('Audios per person (train): ', muestras_aud)
print ('Images per person (test): ', img_test)
print ('Audios per person (test): ', aud_test)


Conocidos:  46
Desconocidos:  46
Muestras por imagen:  40
Muestras por audio:  2
Muestras por imagen test:  30
Muestras por audio test:  1


In [4]:
from sklearn.decomposition import PCA

Data_PCA = Mix_Con_train_embeddings
ShapedData = Data_PCA.reshape(Conocidos*muestras_img,1024)
ShapedData = np.array(ShapedData)

PCA_emb = PCA(n_components=.95)
PCA_emb.fit(ShapedData)
dim = PCA_emb.n_components_

### Start of the experiment

### Funtions for the experiment


In [5]:
def predictor (x,umbral,centroides):    

    distancias = cdist(x, centroides, medida)[0]

    etiquetas = []
    for i in range(len(distancias)):
        if distancias[i]<umbral[i]:
            etiquetas.append(1)
        else:
            etiquetas.append(0)
    
    
# Since is just novelty detection, we only need to know if the sample is known or unknown, not the id

    if sum(etiquetas) == 1:
        #return True,columnas[np.argmax(etiquetas)],distancias # 
        return True,np.argmax(etiquetas),distancias 
    elif sum (etiquetas) > 1:
        #id = columnas[np.argmin(distancias)]
        id = "muchos"
        return True,id,distancias
    else:
        return False,'Desconocido',distancias

    
    

In [6]:
def ensayo (predictor,Data_con,Data_descon,emb_con,emb_descon,muestras_test,umbral,Conocidos_test,centroides):
    
    T_positive = 0 # 
    T_negative = 0 # 
    F_positive = 0 # 
    F_negative = 0 # 
    
    # We take the samples only for the number of known people we want to use in the test.
    

    con_test = Data_con[:muestras_test*Conocidos_test] 
    descon_test = Data_descon
    emb_con_test = emb_con[:muestras_test*Conocidos_test]
    emb_descon_test = emb_descon
    
     # Known test 

    for i in range(len(con_test)):
        [resp,id,dist] = predictor(emb_con_test[i],umbral,centroides)
        if resp == True:
            T_positive += 1
            
        if resp == False:
            F_negative += 1
            
    # Uknonwn test

    
    for i in range(len(descon_test)):
        [resp,id,dist] = predictor(emb_descon_test[i],umbral,centroides)
        if resp == True:
            

            F_positive += 1
        if resp == False:
            T_negative += 1
    
    
    
    
    return T_positive,T_negative,F_positive,F_negative

In [7]:
def calc_umbral(Max_distance,Mean_distance,Mat_dist,pesos):

    umbrales = [pesos[0]* np.mean(Mat_dist[i]) + pesos[1]*np.std(Mat_dist) + 
                pesos[2]*np.mean(Max_distance) + pesos[3]*np.std(Max_distance) + 
                pesos[4]*np.mean(Mean_distance) + pesos[5]*np.std(Mean_distance) for i in range(len(Mat_dist))]
    
    return umbrales

In [8]:
def test_incremental_simple (Conocidos_test,calc_umbral,df,Data_con,Data_descon,emb_con,emb_descon,muestras_test,predictor,ensayo, pesos,long_emb = 512):
    
    # Thresholds and centroids calculation:
    


    Data = df.reshape(Conocidos,len(df[0]),long_emb)
        
    Data = np.array(Data)

    # The centroid of each person is calculated as the mean of all their embeddings.
    

    centroides = np.zeros((Conocidos_test,long_emb))
    for i in range(Conocidos_test):
        centroides[i] = np.mean(Data[i],axis=0)
        
    
    # We compute the distance of all the embeddings of each person with their centroid. We save it in Dist_mat
    
    # With that we calculate the threshold for each person. We save it in umbrales

    Max_distance = []
    Mean_distance = []
    Dist_mat = []
    for i in range (len(centroides)):
        Dist_mat.append(cdist(centroides[i].reshape(1,-1),Data[i],medida)[0])
        Max_distance.append(np.max(cdist(centroides[i].reshape(1,-1),Data[i],medida)))
        Mean_distance.append(np.mean(cdist(centroides[i].reshape(1,-1),Data[i],medida)))
        
    umbral = calc_umbral(Max_distance,Mean_distance,Dist_mat,pesos)

    
    
    T_P, T_N, F_P, F_N = ensayo (predictor,Data_con,Data_descon,emb_con,emb_descon,
                                                                           muestras_test,umbral,Conocidos_test,centroides)
    
    return T_P, T_N, F_P, F_N
    
    

In [9]:
def test_incremental_GMM(Conocidos_test,calc_umbral,df,Data_con,Data_descon,emb_con,emb_descon,muestras_test,predictor,ensayo,pesos,long_emb = 512,N_gauss = 2):
    
    
    Data = df.reshape(Conocidos,len(df[0]),long_emb)

    Data = np.array(Data)
    ShapedData = Data.reshape(Conocidos_test*muestras_img,1024)
    ShapedData = np.array(ShapedData)
    
    N_gauss = N_gauss*Conocidos_test
    GMM_spherical = GaussianMixture(n_components=N_gauss,covariance_type='spherical',max_iter=1000,random_state=0).fit(ShapedData)
    rangos_GMM = GMM_spherical.precisions_
    centroides_GMM = GMM_spherical.means_
    etiquetas_GMM_sph = GMM_spherical.predict(ShapedData)
    covarianzas_GMM = GMM_spherical.covariances_


    etiquetas_persona_GMM_sph = []
    for i in range(Conocidos_test):
        etiquetas_persona_GMM_sph.append([etiquetas_GMM_sph[i*muestras_img + j] for j in range (muestras_img)])
        
    Max_distance_GMM = []
    Mean_distance_GMM = []
    Dist_mat_GMM = []
    
    etiquetas_persona_GMM_sph = np.array(etiquetas_persona_GMM_sph)
    #return etiquetas_persona_GMM_sph
    

    
    for i in range(len(centroides_GMM)):
        
        Persona = np.unique(np.where(etiquetas_persona_GMM_sph == i)[0])[0] # Persona a la que pertenece el centroide i
        

        Dist_mat_GMM.append(cdist (centroides_GMM[i].reshape(1,-1),Data[Persona],metric = medida))
        Max_distance_GMM.append(np.max(Dist_mat_GMM[i]))
        Mean_distance_GMM.append(np.mean(Dist_mat_GMM[i]))
        
    #     etiq_person = np.unique(etiquetas_persona_GMM_sph[i])
    #     centroides_persona = centroides_GMM[etiq_person]
    #     Dist_mat_GMM.append(cdist (centroides_persona,Data[i],metric = medida))
    #     Max_distance_GMM.append(np.max(Dist_mat_GMM[i]))
    #     Mean_distance_GMM.append(np.mean(Dist_mat_GMM[i]))
     
    # for i in range(len(Dist_mat_GMM)):
    #     Dist_actual = np.array(Dist_mat_GMM[i])
    #     N_centroides = Dist_actual.shape[0]
    #     for j in range(N_centroides):
    #         umbrales = calc_umbral(Max_distance_GMM,Mean_distance_GMM,Dist_mat_GMM,modo)
            
            
    umbral = calc_umbral(Max_distance_GMM,Mean_distance_GMM,Dist_mat_GMM,pesos) # Distancias a los umbrales


    T_P, T_N, F_P, F_N = ensayo (predictor,Data_con,Data_descon,emb_con,emb_descon,
                                                                            muestras_test,umbral,Conocidos_test,centroides_GMM)
    
    return T_P, T_N, F_P, F_N

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns

def obt_results (resultados,mustras_test):
    resultados = np.array(resultados)
    
    base = Conocidos*mustras_test
    T_Ps = [(resultados[i][0]/base) for i in range(len(resultados))]
    T_Ns = [(resultados[i][1]/base) for i in range(len(resultados))]
    F_Ps = [(resultados[i][2]/base) for i in range(len(resultados))]
    F_Ns = [(resultados[i][3]/base) for i in range(len(resultados))]
    

    
    return T_Ps,T_Ns,F_Ps,F_Ns

def mostrar_resultados (resultados,muestras_test):
    T_Ps,T_Ns,F_Ps,F_Ns = obt_results (resultados,muestras_test)
    
    x = np.arange(1,len(resultados)+1)
    
    fig,ax = plt.subplots(1,2,figsize=(20,10))
    
    ax[0].bar(x,T_Ps,color="red",alpha=0.5)
    ax[0].plot(x,T_Ps,label="T_Ps")

    ax[1].bar(x,T_Ns,color="red",alpha=0.5)
    ax[1].plot(x,T_Ns,label="T_Ns")
    
    ax[0].set_title("True Positives", fontsize=20)
    ax[0].set_ylabel("% True Positives", fontsize=15)
    ax[0].set_xlabel("Tipo de umbral", fontsize=15)

    ax[1].set_title("True Negatives", fontsize=20)
    ax[1].set_ylabel("% True Negatives", fontsize=15)
    ax[1].set_xlabel("Tipo de umbral", fontsize=15)

    #plt.savefig("Resultados_1.png")
    plt.show()
    
    
    fig,ax = plt.subplots(1,2,figsize=(25,10))
    
    ax[0].bar(x,F_Ps,color="purple",alpha=0.5)
    ax[0].plot(x,F_Ps,label="F_Ps",color="black")
    
    ax[1].bar(x,F_Ns,color="purple",alpha=0.5)
    ax[1].plot(x,F_Ns,label="F_Ns",color="black")

    ax[0].set_title("False Positives", fontsize=20)
    ax[0].set_ylabel("% False Positives", fontsize=15)
    ax[0].set_xlabel("Tipo de umbral", fontsize=15)   

    ax[1].set_title("False Negatives", fontsize=20)
    ax[1].set_ylabel("% False Negatives", fontsize=15)
    ax[1].set_xlabel("Tipo de umbral", fontsize=15)


    #plt.savefig("Resultados_2.png")
    plt.show()
    
    
    

In [11]:

def GA_test(fitness_func,file_name,num_generations = 150,num_parents_mating = 4,sol_per_pop = 20,parent_selection_type = 'sss',keep_parents = 4,
            crossover_type = "two_points",mutation_type = "random",mutation_num_genes = 5,initial_population = None):
    
    def callback_generation(ga_instance): 
        print("Generation = {generation}".format(generation=ga_instance.generations_completed))
        print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution()[1]))
        print ("Best individual = ",ga_instance.best_solution()[0])
        last_fitness = ga_instance.best_solution()[1]
        
    fitness_function = fitness_func

    num_genes = 6# Númerod de genes en los individuos 


    random_mutation_min_val = -5
    random_mutation_max_val = 5
    if initial_population != None:
        initial_population =[initial_population] * sol_per_pop 
    

    
    gene_space = [{'low': -5, 'high': 5,'step':0.01}] * num_genes # Search space
    last_fitness = 0


    ga_instance = pygad.GA(num_generations=num_generations,
                            num_parents_mating=num_parents_mating,
                                fitness_func=fitness_function,
                                sol_per_pop=sol_per_pop,
                                num_genes=num_genes,
                               # init_range_low=init_range_low,
                               # init_range_high=init_range_high,
                                parent_selection_type=parent_selection_type,
                                crossover_type=crossover_type,
                                mutation_type=mutation_type,
                                mutation_num_genes=mutation_num_genes,
                                gene_space=gene_space,
                                allow_duplicate_genes=False,
                                keep_elitism= keep_parents,
                                initial_population=initial_population,
                                on_generation=callback_generation)
        
    ga_instance.run()
    
    save_ga(ga_instance,file_name)
    
    
path_save_ga = 'GA_results/'

In [21]:
def fitness_func(ga,solution,solution_idx):
    results = (test_incremental_simple(Conocidos,calc_umbral,Img_Con_train_embeddings,Img_Con_validation_paths,Img_Descon_validation_paths,
                                       Img_Con_validation_embeddings.reshape(Conocidos*img_test,1,512),Img_Descon_validation_embeddings.reshape(Conocidos*img_test,1,512),30,predictor,ensayo,solution))
    Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen = calculate_metrics(*results)
    
    return Acurracy_imagen
 
GA_test(fitness_func,path_save_ga+ 'GA_imagen_'+set,num_generations = 350,
        sol_per_pop=100,mutation_num_genes = 3,keep_parents=2,parent_selection_type='tournament',initial_population=[0,0,1,-3,0,0])


Generation = 1
Fitness    = 0.9721014492753624
Best individual =  [ 0.   -1.05  1.26  0.14  0.36 -3.55]
Generation = 2
Fitness    = 0.9746376811594203
Best individual =  [ 0.    1.83  1.   -1.08 -0.7   3.78]
Generation = 3
Fitness    = 0.9746376811594203


KeyboardInterrupt: 

In [59]:
resultados = pd.DataFrame(columns=['Precision','Recall','F1','Acurracy','Tipo'])
res = pygad.load(path_save_ga+ 'GA_imagen_'+set)
solution, solution_fitness, solution_idx = res.best_solution()
results = (test_incremental_simple(Conocidos,calc_umbral,Img_Con_train_embeddings,Img_Con_test_paths,Img_Descon_test_paths,
                                        Img_Con_test_embeddings.reshape(Conocidos*img_test,1,512),Img_Descon_test_embeddings.reshape(Conocidos*img_test,1,512),30,predictor,ensayo,solution))

Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen = calculate_metrics(*results)
resultados.loc[0] = [Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen,'Imagen']
print(Acurracy_imagen)
print (results)


0.9804347826086957
(1343, 1363, 17, 37)


In [47]:
def fitness_func(ga,solution,solution_idx):
    results = (test_incremental_simple(Conocidos,calc_umbral,Aud_Con_train_embeddings,Aud_Con_validation_paths,Aud_Descon_validation_paths,
                                       Aud_Con_validation_embeddings.reshape(Conocidos*aud_test,1,512),Aud_Descon_validation_embeddings.reshape(Conocidos*aud_test,1,512),30,predictor,ensayo,solution))
    Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen = calculate_metrics(*results)
    
    return Acurracy_imagen






GA_test(fitness_func,path_save_ga + 'GA_audio_'+set,num_generations = 30)


Generation = 1
Fitness    = 0.8043478260869565
Best individual =  [-2.23  1.69  3.21 -1.61  0.06  4.22]
Generation = 2
Fitness    = 0.8043478260869565
Best individual =  [-2.23  1.69  3.21 -1.61  0.06  4.22]
Generation = 3
Fitness    = 0.8043478260869565
Best individual =  [-2.23  1.69  3.21 -1.61  0.06  4.22]
Generation = 4
Fitness    = 0.8804347826086957
Best individual =  [ 0.61  1.1  -1.73 -1.17  3.15 -0.57]
Generation = 5
Fitness    = 0.8804347826086957
Best individual =  [ 0.61  1.1  -1.73 -1.17  3.15 -0.57]
Generation = 6
Fitness    = 0.8804347826086957
Best individual =  [ 0.61  1.1  -1.73 -1.17  3.15 -0.57]
Generation = 7
Fitness    = 0.8804347826086957
Best individual =  [ 0.61  1.1  -1.73 -1.17  3.15 -0.57]
Generation = 8
Fitness    = 0.8804347826086957
Best individual =  [ 0.61  1.1  -1.73 -1.17  3.15 -0.57]
Generation = 9
Fitness    = 0.8804347826086957
Best individual =  [ 0.61  1.1  -1.73 -1.17  3.15 -0.57]
Generation = 10
Fitness    = 0.8804347826086957
Best individual 

In [60]:
res = pygad.load(path_save_ga+ 'GA_audio_'+set)
solution, solution_fitness, solution_idx = res.best_solution()
results = (test_incremental_simple(Conocidos,calc_umbral,Aud_Con_train_embeddings,Aud_Con_test_paths,Aud_Descon_test_paths,
                                        Aud_Con_test_embeddings.reshape(Conocidos*aud_test,1,512),Aud_Descon_test_embeddings.reshape(Conocidos*aud_test,1,512),30,predictor,ensayo,solution))

Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen = calculate_metrics(*results)

resultados.loc[1] = [Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen,'Audio']
print(Acurracy_imagen)
print (Precision_imagen)
print (results)

0.9239130434782609
0.9148936170212766
(43, 42, 4, 3)


In [18]:
def fitness_func(ga,solution,solution_idx):
    results = (test_incremental_simple(Conocidos,calc_umbral,Mix_Con_train_embeddings,Mix_Con_validation_paths,Mix_Descon_validation_paths,
                                       Mix_Con_validation_embeddings.reshape(Conocidos*img_test,1,1024),Mix_Descon_validation_embeddings.reshape(Conocidos*img_test,1,1024),30,predictor,ensayo,solution,1024))
    Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen = calculate_metrics(*results)
    
    return Acurracy_imagen





GA_test(fitness_func,path_save_ga + 'GA_mixto_'+set,num_generations = 60)

Generation = 1
Fitness    = 0.8985507246376812
Best individual =  [-1.24  4.05 -1.21 -1.09  3.78  1.99]
Generation = 2
Fitness    = 0.9876811594202899
Best individual =  [ 0.82 -4.65  3.89 -2.54 -3.15  1.99]
Generation = 3
Fitness    = 0.9876811594202899
Best individual =  [ 0.82 -4.65  3.89 -2.54 -3.15  1.99]
Generation = 4
Fitness    = 0.9876811594202899
Best individual =  [ 0.82 -4.65  3.89 -2.54 -3.15  1.99]
Generation = 5
Fitness    = 0.9876811594202899
Best individual =  [ 0.82 -4.65  3.89 -2.54 -3.15  1.99]
Generation = 6
Fitness    = 0.9876811594202899
Best individual =  [ 0.82 -4.65  3.89 -2.54 -3.15  1.99]
Generation = 7
Fitness    = 0.9876811594202899
Best individual =  [ 0.82 -4.65  3.89 -2.54 -3.15  1.99]
Generation = 8
Fitness    = 0.9876811594202899
Best individual =  [ 0.82 -4.65  3.89 -2.54 -3.15  1.99]
Generation = 9
Fitness    = 0.9876811594202899
Best individual =  [ 0.82 -4.65  3.89 -2.54 -3.15  1.99]
Generation = 10
Fitness    = 0.9891304347826086
Best individual 

In [61]:
res = pygad.load(path_save_ga+ 'GA_mixto_'+set)
solution, solution_fitness, solution_idx = res.best_solution()

results = (test_incremental_simple(Conocidos,calc_umbral,Mix_Con_train_embeddings,Mix_Con_test_paths,Mix_Descon_test_paths,
                                       Mix_Con_test_embeddings.reshape(Conocidos*img_test,1,1024),Mix_Descon_test_embeddings.reshape(Conocidos*img_test,1,1024),30,predictor,ensayo,solution,1024))
Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen = calculate_metrics(*results)
resultados.loc[2] = [Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen,'Mixto']
print (results)
print (Acurracy_imagen)

(1365, 1379, 1, 15)
0.9942028985507246


In [14]:
PCA_Con_train_embeddings = PCA_emb.transform(Mix_Con_train_embeddings.reshape(Conocidos*muestras_img,1024))
PCA_Con_test_embeddings = PCA_emb.transform(Mix_Con_test_embeddings.reshape(Conocidos*img_test,1024))
PCA_Descon_test_embeddings = PCA_emb.transform(Mix_Descon_test_embeddings.reshape(Conocidos*img_test,1024))
PCA_Con_validation_embeddings = PCA_emb.transform(Mix_Con_validation_embeddings.reshape(Conocidos*img_test,1024))
PCA_Descon_validation_embeddings = PCA_emb.transform(Mix_Descon_validation_embeddings.reshape(Conocidos*img_test,1024))

In [14]:

def fitness_func(ga,solution, solution_idx):
    results = (test_incremental_simple(Conocidos,calc_umbral,PCA_Con_train_embeddings.reshape(Conocidos,muestras_img,dim),Mix_Con_validation_paths,Mix_Descon_validation_paths,
                                       PCA_Con_validation_embeddings.reshape(Conocidos*img_test,1,dim),PCA_Descon_validation_embeddings.reshape(Conocidos*img_test,1,dim),
                                       30,predictor,ensayo,solution,dim))
    Precision,Recall,F1,Acurracy = calculate_metrics(*results)
    
    return Acurracy


GA_test(fitness_func,path_save_ga + 'GA_PCA_'+set,num_generations = 200,sol_per_pop = 60)

Generation = 1
Fitness    = 0.9384057971014492
Best individual =  [-0.41 -4.77  0.83 -4.14  2.54  0.53]
Generation = 2
Fitness    = 0.9384057971014492
Best individual =  [-0.41 -4.77  0.83 -4.14  2.54  0.53]
Generation = 3
Fitness    = 0.9822463768115942
Best individual =  [ 0.13 -3.44  1.48  2.37  0.07 -3.37]
Generation = 4
Fitness    = 0.9822463768115942
Best individual =  [ 0.13 -3.44  1.48  2.37  0.07 -3.37]
Generation = 5
Fitness    = 0.9836956521739131
Best individual =  [ 0.31  1.36  0.19 -4.14  2.3  -3.77]
Generation = 6
Fitness    = 0.9836956521739131
Best individual =  [ 0.31  1.36  0.19 -4.14  2.3  -3.77]
Generation = 7
Fitness    = 0.9836956521739131
Best individual =  [ 0.31  1.36  0.19 -4.14  2.3  -3.77]
Generation = 8
Fitness    = 0.9836956521739131
Best individual =  [ 0.31  1.36  0.19 -4.14  2.3  -3.77]
Generation = 9
Fitness    = 0.9836956521739131
Best individual =  [ 0.31  1.36  0.19 -4.14  2.3  -3.77]
Generation = 10
Fitness    = 0.9836956521739131
Best individual 

In [62]:
res = pygad.load(path_save_ga+ 'GA_PCA_'+set)
solution, solution_fitness, solution_idx = res.best_solution()

results = (test_incremental_simple(Conocidos,calc_umbral,PCA_Con_train_embeddings.reshape(Conocidos,muestras_img,dim),Mix_Con_test_paths,Mix_Descon_test_paths,
                                       PCA_Con_test_embeddings.reshape(Conocidos*img_test,1,dim),PCA_Descon_test_embeddings.reshape(Conocidos*img_test,1,dim),
                                       30,predictor,ensayo,solution,dim))
Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen = calculate_metrics(*results)
resultados.loc[3] = [Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen,'PCA']

print (results)
print (Acurracy_imagen)

(1348, 1349, 31, 32)
0.9771739130434782


In [49]:
def fitness_func(ga,solution, solution_idx):
    results = (test_incremental_GMM(Conocidos,calc_umbral,Mix_Con_train_embeddings,Mix_Con_validation_paths,Mix_Descon_validation_paths,
                                       Mix_Con_validation_embeddings.reshape(Conocidos*img_test,1,1024),Mix_Descon_validation_embeddings.reshape(Conocidos*img_test,1,1024),30,predictor,ensayo,solution,1024))
    Precision,Recall,F1,Acurracy = calculate_metrics(*results)
    
    return Acurracy





GA_test(fitness_func,path_save_ga + 'GA_GMM_'+set,num_generations = 16,initial_population=[-0.36, -2.76,  2.22, -0.08, -0.13,  0.34] )

Generation = 1
Fitness    = 0.9637681159420289
Best individual =  [-0.36 -2.76  2.22 -0.08 -0.13  0.34]
Generation = 2
Fitness    = 0.9637681159420289
Best individual =  [-0.36 -2.76  2.22 -0.08 -0.13  0.34]
Generation = 3
Fitness    = 0.9637681159420289
Best individual =  [-0.36 -2.76  2.22 -0.08 -0.13  0.34]
Generation = 4
Fitness    = 0.9637681159420289
Best individual =  [-0.36 -2.76  2.22 -0.08 -0.13  0.34]
Generation = 5
Fitness    = 0.9637681159420289
Best individual =  [-0.36 -2.76  2.22 -0.08 -0.13  0.34]
Generation = 6
Fitness    = 0.9637681159420289
Best individual =  [-0.36 -2.76  2.22 -0.08 -0.13  0.34]
Generation = 7
Fitness    = 0.9637681159420289
Best individual =  [-0.36 -2.76  2.22 -0.08 -0.13  0.34]
Generation = 8
Fitness    = 0.9637681159420289
Best individual =  [-0.36 -2.76  2.22 -0.08 -0.13  0.34]
Generation = 9
Fitness    = 0.9637681159420289
Best individual =  [-0.36 -2.76  2.22 -0.08 -0.13  0.34]
Generation = 10
Fitness    = 0.9873188405797102
Best individual 

In [63]:
res = pygad.load(path_save_ga+ 'GA_GMM_'+set)
solution, solution_fitness, solution_idx = res.best_solution()    

results = (test_incremental_GMM(Conocidos,calc_umbral,Mix_Con_train_embeddings,Mix_Con_test_paths,Mix_Descon_test_paths,
                                       Mix_Con_test_embeddings.reshape(Conocidos*img_test,1,1024),Mix_Descon_test_embeddings.reshape(Conocidos*img_test,1,1024),30,predictor,ensayo,solution,1024))
Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen = calculate_metrics(*results)
resultados.loc[4] = [Precision_imagen,Recall_imagen,F1_imagen,Acurracy_imagen,'GMM']
print (results)
print (Acurracy_imagen)       

(1364, 1369, 11, 16)
0.9902173913043478


In [64]:
resultados.to_csv('Resultados/resultados_'+set+'GA_centroides.csv',index = False)

In [65]:
resultados.round(4)

,Precision,Recall,F1,Acurracy,Tipo
0,0.9875,0.9732,0.9803,0.9804,Imagen
1,0.9149,0.9348,0.9247,0.9239,Audio
2,0.9993,0.9891,0.9942,0.9942,Mixto
3,0.9775,0.9768,0.9772,0.9772,PCA
4,0.9920,0.9884,0.9902,0.9902,GMM
